In [ ]:
#https://qiita.com/shinmura0/items/811d01384e20bfd1e035

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from keras.layers import Lambda, Input, Dense, Reshape
from keras.models import Model
from keras.datasets import mnist
from keras.datasets import fashion_mnist
from keras.losses import mse
from keras.utils import plot_model
from keras import backend as K
from keras.layers import BatchNormalization, Activation, Flatten
from keras.layers.convolutional import Conv2DTranspose, Conv2D

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import os, datetime
import loadimg
from tqdm import tqdm

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import tensorflow as tf
from keras.backend import tensorflow_backend
config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
session = tf.Session(config=config)
tensorflow_backend.set_session(session)

In [ ]:
SPLIT_SIZE = 32 # multiples of 4
IMG_SIZE = 128

In [ ]:
#8×8のサイズに切り出す
def cut_img(x, number, height=SPLIT_SIZE, width=SPLIT_SIZE):
    print("cutting images ...", flush=True)
    x_out = []
    x_shape = x.shape
    #print("cut_img : ", x_shape)

    for i in tqdm(range(number)):
        shape_0 = np.random.randint(0,x_shape[0])
        shape_1 = np.random.randint(0,x_shape[1]-height)
        shape_2 = np.random.randint(0,x_shape[2]-width)
        temp = x[shape_0, shape_1:shape_1+height, shape_2:shape_2+width]
        #print("cut_img temp : ", temp.shape)
        x_out.append(temp.reshape((height, width, x_shape[3])))

    x_out = np.array(x_out)
    print("Complete.", x_out.shape, flush=True)

    return x_out

In [ ]:
# reparameterization trick
# instead of sampling from Q(z|X), sample eps = N(0,I)
# z = z_mean + sqrt(var)*eps
def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean=0 and std=1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

In [ ]:
GRAY = False
X_TRAIN_PATH = './models/x_train.npy'
X_TEST_PATH = './models/x_test.npy'
X_SPLIT_PATH = './models/x_train_split.npy'

try:
    x_train = np.load(X_TRAIN_PATH)
    x_test = np.load(X_TEST_PATH)
    x_train_split = np.load(X_SPLIT_PATH)
    
#except KeyboardInterrupt:
except FileNotFoundError:
    x_train, _, _, _, _ = loadimg.loadimg('images/AnomalyDetectionPictures/normal',
                                                           imgsize=IMG_SIZE,
                                                           isdirs=False,
                                                           normalize=True,
                                                           onehot=True,
                                                           grayscale=GRAY,
                                                           train_ratio=1)
    np.save(X_TRAIN_PATH, x_train)
    
    _, x_test, _, _, _ = loadimg.loadimg('images/AnomalyDetectionPictures/anomaly',
                                                           imgsize=IMG_SIZE,
                                                           isdirs=False,
                                                           normalize=True,
                                                           onehot=True,
                                                           grayscale=GRAY,
                                                           train_ratio=0)
    np.save(X_TEST_PATH, x_test)
    
    x_train_split = cut_img(x_train, 1000000)
    np.save(X_SPLIT_PATH, x_train_split)
    
print("x_train", x_train.shape)
print("x_test", x_test.shape)
print("x_train_split", x_train_split.shape)

In [ ]:
# Show split images
#plt.figure()

for i in range(40):
    img = x_train_split[i]
    if (img.shape[2] == 1): # Gray scale
        img = img.reshape((img.shape[0], img.shape[1]))
        img = np.stack((img,img,img), axis=-1)
    #plt.subplot(4, 4, i+1)
    plt.imshow(img)
    plt.show()
#plt.close()

In [ ]:
# network parameters
input_shape=(SPLIT_SIZE, SPLIT_SIZE, x_train_split.shape[3])
batch_size = 3000
latent_dim = SPLIT_SIZE//4
epochs = 10
Nc = 16

In [ ]:
# build encoder model
inputs = Input(shape=input_shape, name='encoder_input') 
x = Conv2D(Nc, kernel_size=2, strides=2)(inputs) 
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Conv2D(2*Nc, kernel_size=2, strides=2)(x) 
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Flatten()(x)

z_mean = Dense(latent_dim, name='z_mean')(x)
z_log_var = Dense(latent_dim, name='z_log_var')(x)
z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')
encoder.summary()

In [ ]:
# build decoder model
latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
x = Dense(latent_dim*latent_dim)(latent_inputs)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Reshape((latent_dim,latent_dim,1))(x)
x = Conv2DTranspose(2*Nc, kernel_size=2, strides=2, padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Conv2DTranspose(Nc, kernel_size=2, strides=2, padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x1 = Conv2DTranspose(input_shape[2], kernel_size=4, padding='same')(x)
x1 = BatchNormalization()(x1)
out1 = Activation('sigmoid')(x1)#out.shape=(n,28,28,1)

x2 = Conv2DTranspose(input_shape[2], kernel_size=4, padding='same')(x)
x2 = BatchNormalization()(x2)
out2 = Activation('sigmoid')(x2)#out.shape=(n,28,28,1)

decoder = Model(latent_inputs, [out1, out2], name='decoder')
decoder.summary()

In [ ]:
# build VAE model
outputs_mu, outputs_sigma_2 = decoder(encoder(inputs)[2])
vae = Model(inputs, [outputs_mu, outputs_sigma_2], name='vae_mlp')

# VAE loss
m_vae_loss = (K.flatten(inputs) - K.flatten(outputs_mu))**2 / K.flatten(outputs_sigma_2)
m_vae_loss = 0.5 * K.sum(m_vae_loss)

a_vae_loss = K.log(2 * 3.14 * K.flatten(outputs_sigma_2))
a_vae_loss = 0.5 * K.sum(a_vae_loss)

kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5

vae_loss = K.mean(kl_loss + m_vae_loss + a_vae_loss)
vae.add_loss(vae_loss)
vae.compile(optimizer='adam')

In [ ]:
# train the autoencoder
#vae.fit(x_train_1,
vae.fit(x_train_split,
        epochs=epochs,
        batch_size=batch_size)
        #validation_data=(x_test, None))
now = datetime.datetime.now()
vae.save_weights("models/vae_mnist_{0:%Y%m%d_%H%M%S}.h5".format(now))

In [ ]:
#ヒートマップの描画
def save_img(x_normal, x_anomaly, img_normal, img_anomaly, name):
    path = 'images/'
    if not os.path.exists(path):
          os.mkdir(path)

    #　※注意　評価したヒートマップを1～10に正規化
    img_max = np.max([img_normal, img_anomaly])
    img_min = np.min([img_normal, img_anomaly])
    img_normal = (img_normal-img_min)/(img_max-img_min) * 9 + 1
    img_anomaly = (img_anomaly-img_min)/(img_max-img_min) * 9 + 1

    plt.figure()
    plt.subplot(2, 2, 1)
    #plt.imshow(x_normal[0,:,:,0], cmap='gray')
    plt.imshow(x_normal[0])
    plt.axis('off')
    plt.colorbar()

    plt.subplot(2, 2, 2)
    plt.imshow(img_normal[0,:,:,0], cmap='Blues',norm=colors.LogNorm())
    plt.axis('off')
    plt.colorbar()
    plt.clim(1, 10)

    plt.title(name + "normal")

    plt.subplot(2, 2, 3)
    #plt.imshow(x_anomaly[0,:,:,0], cmap='gray')
    plt.imshow(x_anomaly[0])
    plt.axis('off')
    plt.colorbar()

    plt.subplot(2, 2, 4)
    plt.imshow(img_anomaly[0,:,:,0], cmap='Blues',norm=colors.LogNorm())
    plt.axis('off')
    plt.colorbar()
    plt.clim(1, 10)

    plt.title(name + "anomaly")

    #plt.savefig(path + name +".png")
    plt.show()
    plt.close()

In [ ]:
#ヒートマップの計算
MOVE = 8
def evaluate_img(model, x_normal, x_anomaly, name, height=SPLIT_SIZE, width=SPLIT_SIZE, move=MOVE):
    x_normal = np.asarray([x_normal])
    x_anomaly = np.asarray([x_anomaly])
    img_normal = np.zeros((x_normal.shape))
    img_anomaly = np.zeros((x_normal.shape))
    for i in range(int((x_normal.shape[1]-height)/move)):
        for j in range(int((x_normal.shape[2]-width)/move)):
            x_sub_normal = x_normal[0, i*move:i*move+height, j*move:j*move+width]
            x_sub_anomaly = x_anomaly[0, i*move:i*move+height, j*move:j*move+width]
            x_sub_normal = x_sub_normal.reshape(1, height, width, input_shape[2])
            x_sub_anomaly = x_sub_anomaly.reshape(1, height, width, input_shape[2])

            #従来手法
            if name == "old_":
                #正常のスコア
                normal_score = model.evaluate(x_sub_normal, batch_size=1, verbose=0)
                img_normal[0, i*move:i*move+height, j*move:j*move+width, 0] +=  normal_score

                #異常のスコア
                anomaly_score = model.evaluate(x_sub_anomaly, batch_size=1, verbose=0)
                img_anomaly[0, i*move:i*move+height, j*move:j*move+width, 0] +=  anomaly_score

            #提案手法
            else:
                #正常のスコア
                mu, sigma = model.predict(x_sub_normal, batch_size=1, verbose=0)
                loss = 0
                for k in range(height):
                    for l in range(width):
                        for m in range(input_shape[2]):
                            loss += 0.5 * (x_sub_normal[0,k,l,m] - mu[0,k,l,m])**2 / sigma[0,k,l,m]
                img_normal[0, i*move:i*move+height, j*move:j*move+width] +=  loss

                #異常のスコア
                mu, sigma = model.predict(x_sub_anomaly, batch_size=1, verbose=0)
                loss = 0
                for k in range(height):
                    for l in range(width):
                        for m in range(input_shape[2]):
                            loss += 0.5 * (x_sub_anomaly[0,k,l,m] - mu[0,k,l,m])**2 / sigma[0,k,l,m]
                img_anomaly[0, i*move:i*move+height, j*move:j*move+width] +=  loss

    save_img(x_normal, x_anomaly, img_normal, img_anomaly, name)

In [ ]:
train = x_train[0]
for test in x_test:
    #evaluate_img(vae, [x_train[i]], [x_test[i]], "old_")
    evaluate_img(vae, train, test, "new_")